In [1]:
import numpy as np
from models import create_mlp_model
from cross_validation import CV_Skeleton

Using TensorFlow backend.


### Load iris data set just for test.

In [2]:
from sklearn.datasets import load_iris
from keras.utils import to_categorical

iris = load_iris()
X = iris['data']
y = iris['target']
# create the sparse target
y_sparse = to_categorical(y)

names = iris['target_names']
feature_names = iris['feature_names']

In [3]:
y_sparse[:3]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

### Create the cross-validation.


In [4]:
from sklearn.model_selection import StratifiedKFold

n_folds = 10

skf = StratifiedKFold(n_splits=n_folds, random_state=17, shuffle=True)

print(skf)

StratifiedKFold(n_splits=10, random_state=17, shuffle=True)


### Create the preproc method

In [5]:
from sklearn.preprocessing import StandardScaler

scl_factor = StandardScaler()

print(scl_factor)

StandardScaler(copy=True, with_mean=True, with_std=True)


### Create the keras model

In [6]:
hidden_layer_neurons      = 5
output_layer_neurons      = 3
number_of_hidden_layers   = 1
hidden_layer_act_function = 'relu'
output_layer_act_function = 'softmax'
loss_function             = 'categorical_crossentropy'
optimizer                 = 'adam'

In [7]:
cv_test_model = create_mlp_model(input_dim=X.shape[1], output_dim=output_layer_neurons,
                         n_neurons=hidden_layer_neurons, n_layers=number_of_hidden_layers,
                         hl_act_func=hidden_layer_act_function,
                         ol_act_func=output_layer_act_function,
                         loss_func=loss_function,
                         optimizer=optimizer,
                         name='test_model'
                        )

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
print(cv_test_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________
None


### Adjust some tuning parameters

In [9]:
t_params = {'n_inits'   : 2,
           'batch_size' : 50,
           'epochs'     : 10}

In [10]:
cv_test = CV_Skeleton(X, y, 
                      tuning_params=t_params,
                      preproc_method=scl_factor,
                      cv_method=skf,
                      ml_model=cv_test_model)

### Now we make the cross-validation loop

In [11]:
cv_test.cv_loop()

Training in the Fold: 1 | Init: 1
Instructions for updating:
Use tf.cast instead.
Train on 135 samples, validate on 15 samples
Epoch 1/10
135/135 [==============================] - 0s 3ms/step - loss: 0.9463 - acc: 0.5778 - val_loss: 0.8899 - val_acc: 0.7333
Epoch 2/10
135/135 [==============================] - 0s 57us/step - loss: 0.9393 - acc: 0.5926 - val_loss: 0.8830 - val_acc: 0.7333
Epoch 3/10
135/135 [==============================] - 0s 127us/step - loss: 0.9320 - acc: 0.5926 - val_loss: 0.8758 - val_acc: 0.7333
Epoch 4/10
135/135 [==============================] - 0s 126us/step - loss: 0.9246 - acc: 0.6000 - val_loss: 0.8684 - val_acc: 0.7333
Epoch 5/10
135/135 [==============================] - 0s 138us/step - loss: 0.9172 - acc: 0.6222 - val_loss: 0.8611 - val_acc: 0.7333
Epoch 6/10
135/135 [==============================] - 0s 122us/step - loss: 0.9099 - acc: 0.6222 - val_loss: 0.8538 - val_acc: 0.7333
Epoch 7/10
135/135 [==============================] - 0s 83us/step - los

In [12]:
# get the results of cross-validation
result_dict = cv_test.get_cv_dict()

In [13]:
result_dict[0].keys()

dict_keys(['trn_tst_indices', 'history', 'weights'])

In [14]:
result_dict[0]['weights'], result_dict[1]['weights']

([array([[-0.7607583 , -0.66369814,  0.30523407, -0.11777232,  0.5637772 ],
         [-0.68594587, -0.48333204, -0.3476057 , -0.5836633 ,  0.56206644],
         [-0.27662924, -0.00189115, -0.7020519 ,  0.24855244, -0.03670489],
         [ 0.7339188 ,  0.41540158,  0.61945623,  0.837243  , -0.26379085]],
        dtype=float32),
  array([-0.02194587, -0.02185101,  0.02808422,  0.02867591,  0.02369108],
        dtype=float32),
  array([[ 0.7709994 ,  0.05767832, -0.61621904],
         [ 0.35208693, -0.55146855, -0.09618444],
         [-0.6273001 ,  0.83503383,  0.73974013],
         [-0.8514454 ,  0.73947996,  0.6853306 ],
         [ 0.66307116, -0.43978062,  0.41644332]], dtype=float32),
  array([ 0.01511719, -0.00371381, -0.01324048], dtype=float32)],
 [array([[-0.10705221,  0.01503872, -0.44918078,  0.39048067, -0.596681  ],
         [ 0.09136633, -0.24690545,  0.40919012, -0.3970066 ,  0.60898423],
         [-0.54943   , -0.2664579 , -0.0692044 , -0.7053366 ,  0.7464758 ],
         [-